## Concepts

From one current user find best correlated user

Recommend 5 best rated anime from the best correlated user

Use the best rated anime from current user and recommend 5 more 

Total 10 recommendation is provided 

## Import Library

In [146]:
#pip install pandas
import pandas as pd
import string
import re

## Variables

In [147]:
target_user_id = 75002
target_vote_number = 2000
min_vote_number = 1000
numberOfResult = 5

## Loading Dataset
▪ Loading anime.csv and rating.csv into DataFrame

In [148]:
anime_data = pd.read_csv('anime.csv')
#animeDf = animeDf[:5000]
anime_data = pd.DataFrame(anime_data)
anime_data

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [149]:
rating_data = pd.read_csv('rating.csv')
#rating_data = rating_data[:1000000]
new_rating_data = pd.read_csv('newRating.csv')
rating_data = pd.DataFrame(rating_data)
rating_data = pd.concat([rating_data, new_rating_data])
rating_data

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
75,75005,10620,9
76,75005,22319,10
77,75005,15689,10
78,75005,33286,4


# Data Preprocessing

## Change variable type

In [150]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [151]:
anime_data['episodes'] = anime_data['episodes'].replace('Unknown', -1)
anime_data = anime_data.astype({"name":"string","genre":"string","type":"string","episodes":"int64"})

In [152]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  string 
 2   genre     12232 non-null  string 
 3   type      12269 non-null  string 
 4   episodes  12294 non-null  int64  
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(3), string(3)
memory usage: 672.5 KB


In [153]:
rating_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7813817 entries, 0 to 79
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 238.5 MB


## Rename variable

In [154]:
anime_data = anime_data.rename(columns={"rating": "average_rating"})
anime_data.head(3)

,anime_id,name,genre,type,episodes,average_rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262


In [155]:
rating_data = rating_data.rename(columns={"rating": "user_rating"})
rating_data.head(3)

,user_id,anime_id,user_rating
0,1,20,-1
1,1,24,-1
2,1,79,-1


## Check Missing Value

In [156]:
rating_data.isna().sum()

user_id        0
anime_id       0
user_rating    0
dtype: int64

In [157]:
anime_data.isna().sum()

anime_id            0
name                0
genre              62
type               25
episodes            0
average_rating    230
members             0
dtype: int64

## Drop Missing Value

In [158]:
anime_data.dropna(axis=0, inplace= True)
anime_data.isna().sum()

anime_id          0
name              0
genre             0
type              0
episodes          0
average_rating    0
members           0
dtype: int64

## Check Duplicates

In [159]:
#check duplicate anime
duplicated_anime = anime_data[anime_data.duplicated()].shape[0]
print(f'count of duplicated anime: {duplicated_anime}')

count of duplicated anime: 0


In [160]:
#check duplicate rating
duplicated_rating = rating_data[rating_data.duplicated()].shape[0]
print(f'count of duplicated rating: {duplicated_rating}')

count of duplicated rating: 1


In [161]:
rating_data.drop_duplicates(keep='first',inplace=True)
duplicated_rating = rating_data[rating_data.duplicated()].shape[0]
print(f'count of duplicated rating after removing: {duplicated_rating}')

count of duplicated rating after removing: 0


## Merge Dataset by anime_id

In [162]:
raw_merged_data = pd.merge(rating_data, anime_data, on='anime_id', sort = True)
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
...,...,...,...,...,...,...,...,...,...
7813684,53492,34475,5,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813685,70547,34475,9,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813686,70768,34475,-1,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813687,48766,34476,-1,Platonic Chain: Ansatsu Jikkouchuu,"Sci-Fi, Slice of Life",Special,1,1.67,51


## Drop Unnessary Column

In [163]:
#members are not nessesary
#raw_merged_data = raw_merged_data.drop('members', axis=1)
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
...,...,...,...,...,...,...,...,...,...
7813684,53492,34475,5,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813685,70547,34475,9,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813686,70768,34475,-1,Monster Strike: Rain of Memories,"Action, Fantasy, Game",ONA,1,6.58,313
7813687,48766,34476,-1,Platonic Chain: Ansatsu Jikkouchuu,"Sci-Fi, Slice of Life",Special,1,1.67,51


## Drop Unnessary Row
Drop the row that less than min_vote_number

In [164]:
#get number of rating for each anime
grouped_data = pd.DataFrame(raw_merged_data.groupby('anime_id')['user_rating'].count())
#get the anime that fulfilled the constaint which is more than min_vote_number
trimmed_data = grouped_data[grouped_data['user_rating'] > min_vote_number]
#trim the data
raw_merged_data = raw_merged_data.loc[raw_merged_data['anime_id'].isin(trimmed_data.index.values.tolist())]
raw_merged_data

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
0,13,1,-1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
1,19,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
2,21,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
3,23,1,9,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
4,32,1,10,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,8.82,486824
...,...,...,...,...,...,...,...,...,...
7813606,73188,34240,8,Shelter,"Music, Sci-Fi",Music,1,8.38,71136
7813607,73247,34240,10,Shelter,"Music, Sci-Fi",Music,1,8.38,71136
7813608,73257,34240,8,Shelter,"Music, Sci-Fi",Music,1,8.38,71136
7813609,73319,34240,9,Shelter,"Music, Sci-Fi",Music,1,8.38,71136


## Clean Text

In [165]:
#before clean text
pd.DataFrame(anime_data.loc[anime_data['anime_id'] == 9969])

,anime_id,name,genre,type,episodes,average_rating,members
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [166]:
def clean_text(text):
    # remove sybmols and other words
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'&quot;', '', text)
    text = re.sub(r'.hack//', '', text)
    text = re.sub(r'&#039;', '', text)
    text = re.sub(r'A&#039;s', '', text)
    text = re.sub(r'I&#039;', 'I\'', text)
    text = re.sub(r'&amp;', 'and', text)
    #text = re.sub('[^a-zA-Z0-9\s]','', text)
  
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [167]:
raw_merged_data.loc[:, 'name']=raw_merged_data['name'].apply(clean_text)
anime_data.loc[:, 'name'] = anime_data['name'].apply(clean_text)

In [168]:
#after clean text 28977 9969 15417
pd.DataFrame(anime_data.loc[anime_data['anime_id'] == 9969])

,anime_id,name,genre,type,episodes,average_rating,members
4,9969,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


# PART 1: User Based Correlation

## Find rating_anime_matrix

In [169]:
rating_anime_matrix = raw_merged_data.pivot_table(index='user_id', columns=['anime_id'], values = 'user_rating', fill_value= -1)
rating_anime_matrix.head()

anime_id,1,5,6,7,15,16,18,19,20,22,...,32379,32380,32438,32542,32729,32828,32935,32998,33028,34240
user_id,,,,,,,,,,,,,,,,,,,,,
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,8,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,8,-1,6,-1,6,-1,6,5,...,-1,-1,-1,-1,-1,7,-1,-1,-1,-1


## Find anime_rating_matrix

In [170]:
anime_rating_matrix = rating_anime_matrix.transpose()
anime_rating_matrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,73513,73514,73515,73516,75000,75001,75002,75003,75004,75005
anime_id,,,,,,,,,,,,,,,,,,,,,
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,9.0,-1.0,10.0,-1.0,9.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,8.0,-1.0,10.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,-1.0,-1.0,-1.0,-1.0,8.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,10.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
15,-1.0,-1.0,-1.0,-1.0,6.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


## Find the information of target_user_id

In [171]:
anime_rating_user = anime_rating_matrix[target_user_id]
pd.DataFrame(anime_rating_user.sort_values(ascending= False)).rename(columns={target_user_id: f"user_id {target_user_id}'s rating"})

,user_id 75002's rating
anime_id,
23273,10.0
4181,10.0
6547,9.0
4224,9.0
9989,9.0
...,...
2476,-1.0
2472,-1.0
2424,-1.0


## Find Correlation between target_user_id with all the other users

In [172]:
similar_users = anime_rating_matrix.corrwith(anime_rating_user).dropna()
similar_users

user_id
1       -0.003671
2       -0.001834
3        0.070902
5        0.069143
7        0.017921
           ...   
75001   -0.004853
75002    1.000000
75003    0.518078
75004    0.181307
75005   -0.006327
Length: 69380, dtype: float64

## Format into Data Frame

In [173]:
sorted_similar_users = pd.DataFrame(similar_users, columns=['correlation']).sort_values(ascending= False, by= 'correlation')
sorted_similar_users

,correlation
user_id,
75002,1.000000
71911,0.571906
9206,0.524979
75003,0.518078
8630,0.511384
...,...
5492,-0.039678
59486,-0.040336
59531,-0.042284


## Find the best correlated user

In [174]:
#most similar user is the user that at index number 1
most_similar_user = sorted_similar_users.index.values
most_similar_user = most_similar_user[1]
most_similar_user

71911

## Find 5 best rated anime from the user that have highest correlation value

In [175]:
#sort value by rating
target_user = raw_merged_data.loc[raw_merged_data['user_id'] == target_user_id].sort_values(by= 'user_rating', ascending= False)

user_recommendation = raw_merged_data[raw_merged_data['user_id'] == most_similar_user]
user_recommendation = user_recommendation.sort_values(by= 'user_rating', ascending= False)

#Trim the result exist in user's rating
user_recommendation = user_recommendation[~user_recommendation['anime_id'].isin(target_user['anime_id'].tolist())]
user_recommendation = user_recommendation[:numberOfResult]
user_recommendation

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
5836894,71911,13759,10,Sakurasou no Pet na Kanojo,"Comedy, Drama, Romance, School, Slice of Life",TV,24,8.40,380375
6670940,71911,19815,10,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",TV,12,8.47,602291
6742943,71911,20583,10,Haikyuu,"Comedy, Drama, School, Shounen, Sports",TV,25,8.68,284498
4110556,71911,6746,9,Durarara,"Action, Mystery, Supernatural",TV,24,8.38,556431
5504564,71911,11757,9,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100


In [176]:
user_recommendation_list = user_recommendation['anime_id'].tolist()
user_recommendation_list

[13759, 19815, 20583, 6746, 11757]

# PART 2: Item Based Correlation

## Use the user's highest rated anime to recommend other related anime

In [177]:
target_user_rating = rating_data.loc[rating_data['user_id'] == target_user_id].sort_values(by = 'user_rating', ascending= False)
pd.DataFrame(target_user_rating.head(10))

,user_id,anime_id,user_rating
28,75002,4181,10
30,75002,23273,10
39,75002,33352,10
29,75002,28851,9
34,75002,9989,9
35,75002,4224,9
36,75002,6547,9
37,75002,31646,9
27,75002,32281,8
32,75002,31043,8


In [178]:
top_rated_anime_id = target_user_rating['anime_id'].tolist()[0]
top_rated_anime_id

4181

In [179]:
ratings = rating_anime_matrix[top_rated_anime_id]
pd.DataFrame(ratings).rename(columns={top_rated_anime_id: f"{top_rated_anime_id}'s rating"})

,4181's rating
user_id,
1,-1
2,-1
3,-1
4,-1
5,-1
...,...
75001,-1
75002,10
75003,10


In [180]:
similar_animes = rating_anime_matrix.corrwith(ratings)
pd.DataFrame(similar_animes, columns=['correlation'])

,correlation
anime_id,
1,0.120990
5,0.064449
6,0.095167
7,0.027163
15,0.075823
...,...
32828,0.106856
32935,0.077343
32998,0.079204


## Identify the most correlated anime

In [181]:
sorted_similar_anime = pd.DataFrame(similar_animes, columns=['correlation']).sort_values(by= 'correlation', ascending= False)
sorted_similar_anime

,correlation
anime_id,
4181,1.000000
2167,0.840837
4059,0.577209
6351,0.574653
6547,0.421587
...,...
3375,0.004717
746,0.002128
553,-0.001170


## Eliminate the source anime

In [182]:
sorted_similar_anime = sorted_similar_anime[1:]
sorted_similar_anime

,correlation
anime_id,
2167,0.840837
4059,0.577209
6351,0.574653
6547,0.421587
4224,0.413479
...,...
3375,0.004717
746,0.002128
553,-0.001170


## Ensure the identified anime is popular
Higher number of votes means more popular

In [183]:
#Get number of rating for each anime
rating_votes = pd.DataFrame(raw_merged_data.groupby('anime_id')['user_rating'].count())
rating_votes.rename(columns={'user_rating': 'rating_count'})

,rating_count
anime_id,
1,15510
5,6927
6,11077
7,2629
15,2424
...,...
32828,1467
32935,1234
32998,1825


In [184]:
similar_animes_ratings = sorted_similar_anime.join(rating_votes['user_rating']).sort_values(by = 'correlation', ascending = False)
similar_animes_ratings

,correlation,user_rating
anime_id,,
2167,0.840837,21722
4059,0.577209,9402
6351,0.574653,7911
6547,0.421587,27185
4224,0.413479,24285
...,...,...
3375,0.004717,1856
746,0.002128,1936
553,-0.001170,1671


## Get the anime that have higher votes and have higher correlation

In [185]:
similar_popular_animes = similar_animes_ratings.loc[similar_animes_ratings['user_rating']>=target_vote_number].dropna()
similar_popular_animes.head()

,correlation,user_rating
anime_id,,
2167,0.840837,21722
4059,0.577209,9402
6351,0.574653,7911
6547,0.421587,27185
4224,0.413479,24285


In [186]:
#trim the result exist in user's rating
similar_popular_animes = similar_popular_animes[~similar_popular_animes.index.isin(target_user['anime_id'].tolist())]
similar_popular_animes.head()

,correlation,user_rating
anime_id,,
2167,0.840837,21722
4059,0.577209,9402
6351,0.574653,7911
1530,0.412801,7268
1723,0.401586,5102


In [187]:
#make it a list
most_similar_popular_animes = similar_popular_animes[:numberOfResult]
most_similar_popular_animes_list = most_similar_popular_animes.index.to_list()
most_similar_popular_animes_list

[2167, 4059, 6351, 1530, 1723]

In [188]:
user_recommendation = raw_merged_data[raw_merged_data['user_id'] == most_similar_user]
user_recommendation = user_recommendation.sort_values(by= 'user_rating', ascending= False)
user_recommendation = user_recommendation[:numberOfResult]

# Result Showing

In [189]:
final_recommendation_result = user_recommendation_list.copy()
final_recommendation_result += most_similar_popular_animes_list.copy()
final_recommendation_result

[13759, 19815, 20583, 6746, 11757, 2167, 4059, 6351, 1530, 1723]

## Show Target User's Rating

In [190]:
#target_user = raw_merged_data.loc[raw_merged_data['user_id'] == target_user_id].sort_values(by= 'user_rating', ascending= False)
pd.DataFrame(target_user)

,user_id,anime_id,user_rating,name,genre,type,episodes,average_rating,members
3289276,75002,4181,10,Clannad After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24,9.06,456749
7073223,75002,23273,10,Shigatsu wa Kimi no Uso,"Drama, Music, Romance, School, Shounen",TV,22,8.92,416397
3341279,75002,4224,9,Toradora,"Comedy, Romance, School, Slice of Life",TV,25,8.45,633817
4017609,75002,6547,9,Angel Beats,"Action, Comedy, Drama, School, Supernatural",TV,13,8.39,717796
4919134,75002,9989,9,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,"Drama, Slice of Life, Supernatural",TV,11,8.62,463835
5397199,75002,11553,8,Toradora Bentou no Gokui,"Comedy, Romance, School, Slice of Life",Special,1,7.54,71433
7640097,75002,31043,8,Boku dake ga Inai Machi,"Mystery, Psychological, Seinen, Supernatural",TV,12,8.65,402381
7767663,75002,32281,8,Kimi no Na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
4268646,75002,7311,7,Suzumiya Haruhi no Shoushitsu,"Comedy, Mystery, Romance, School, Sci-Fi, Supe...",Movie,1,8.81,240297
7663176,75002,31240,7,ReZero kara Hajimeru Isekai Seikatsu,"Drama, Fantasy, Psychological, Thriller",TV,25,8.64,355839


## Show Target Anime's information 

In [191]:
target_search_anime = anime_data.loc[anime_data['anime_id'] == top_rated_anime_id]
pd.DataFrame(target_search_anime)

,anime_id,name,genre,type,episodes,average_rating,members
10,4181,Clannad After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24,9.06,456749


## Show Final Recommendation Result

In [192]:
#Drop the duplicated anime
recommended_anime = anime_data.loc[anime_data['anime_id'].isin(final_recommendation_result)]
recommended_anime.drop_duplicates(subset=['anime_id'], keep='first')

pd.DataFrame(recommended_anime)

,anime_id,name,genre,type,episodes,average_rating,members
43,20583,Haikyuu,"Comedy, Drama, School, Shounen, Sports",TV,25,8.68,284498
118,19815,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",TV,12,8.47,602291
158,13759,Sakurasou no Pet na Kanojo,"Comedy, Drama, Romance, School, Slice of Life",TV,24,8.40,380375
166,6746,Durarara,"Action, Mystery, Supernatural",TV,24,8.38,556431
223,2167,Clannad,"Comedy, Drama, Romance, School, Slice of Life,...",TV,23,8.30,566690
337,1530,Kanon 2006,"Drama, Romance, Slice of Life, Supernatural",TV,24,8.17,163171
370,4059,Clannad Mou Hitotsu no Sekai Tomoyohen,"Drama, Romance, School, Slice of Life",Special,1,8.14,160423
504,6351,Clannad After Story Mou Hitotsu no Sekai Kyouhen,"Drama, Romance, School",Special,1,8.02,138364
804,11757,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100
2103,1723,Clannad Movie,"Drama, Fantasy, Romance, School",Movie,1,7.35,99506
